# Extract Topics and Short desc

In [1]:
!pip install openai langchain 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tiktoken


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain



# Langchain Loaders:
from langchain.document_loaders import YoutubeLoader
from langchain.document_loaders import WebBaseLoader

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
#import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
!pip install bs4


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:

llm3 = ChatOpenAI(temperature=0,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )


In [39]:

website_loader = WebBaseLoader(["https://news.mit.edu/topic/artificial-intelligence2", "https://www.theguardian.com/technology/artificialintelligenceai"])
website_data = website_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=2200)
website_docs = text_splitter.split_documents(website_data)
print (f"You have {len(website_docs)} docs. First doc is {llm3.get_num_tokens(website_docs[0].page_content)} tokens")

print(website_docs[1].metadata)

You have 3 docs. First doc is 1367 tokens
{'source': 'https://www.theguardian.com/technology/artificialintelligenceai', 'title': 'Artificial intelligence (AI) | Technology | The Guardian', 'language': 'en'}


In [7]:
!python -m pip install htmldate


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from htmldate import find_date
import requests
html=requests.get('https://news.mit.edu/topic/artificial-intelligence2').content.decode('utf-8')
date = find_date(html)
print(date)

2023-09-14


In [40]:

# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# - Revenge Against The Spam Calls: A couple of businesses focused on protecting consumers: RoboCall, TrueCaller, DoNotPay, FitIt
# - Wildcard CEOs vs. Prudent CEOs: However, Munger likes to surround himself with prudent CEO's and says he would never hire Musk.
# - Chess Business: Priyav, a college student, expressed his doubts on the MFM Facebook group about his Chess training business, mychesstutor.com, making $12.5K MRR with 90 enrolled.
# - Restaurant Refiller: An MFM Facebook group member commented on how they pay AirMark $1,000/month for toilet paper and toilet cover refills for their restaurant. Shaan sees an opportunity here for anyone wanting to compete against AirMark.
# - Collecting: Shaan shared an idea to build a mobile only marketplace for a collectors' category; similar to what StockX does for premium sneakers.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve distinct topics discussed from many websites' content
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the websites' content
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
    - Topic 2 title: topic 2 description
    - Topic 3 title: topic 3 description
    
- Ignore topics on policy and regulations
- Do not respond with anything outside of the webstes' content. If you can't extract any topics at all in the whole content, say 'Sorry, No topics found in the given content'
- Only pull topics from the websites' content. Do not use the examples
- If the authors' names were mentioned in the transcript, instead of saying 'The Author' refer to the names.
- Make your titles descriptive but concise. Example: 'Shaan's Experience at Twitch' should be 'Shaan's Interesting Projects At Twitch'
- A topic should be substantial, more than just a one-off comment

"""
#- Do not respond with anything outside of the transcript. If you don't see any topics, say, 'No Topics'
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Websites' Content: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

In [41]:
# % START OF EXAMPLES
# - Sam's Elisabeth Murdoch Story: Sam got a call from Elizabeth Murdoch when he had just launched The Hustle. She wanted to generate video content.
# - Shaan's Rupert Murdoch Story: When Shaan was running Blab he was invited to an event organized by Rupert Murdoch during CES in Las Vegas.
# % END OF EXAMPLES

template="""
You are a helpful assistant that helps retrieve topics discussed in websites' content
- You will be given a series of bullet topics of topics found
- Your goal is to exract the topic names and brief 1-sentence description of the topic
- Do not respond with numbers, just bullet points of all topics listed under each other.
- Deduplicate any bullet points you see
- If you think two or more topics are similar and can be merged, merge them together with one topic title and create a new description that fits the merged topics
- Only pull topics from the websites' content. Do not use the examples.
"""
system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Websites' Content: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_combine = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])



In [42]:
chain = load_summarize_chain(llm3,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,
                             combine_prompt=chat_prompt_combine,
                            verbose=True
                            )

In [43]:
topics_found = chain.run({"input_documents": website_docs})



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant that helps retrieve distinct topics discussed from many websites' content
- Your goal is to extract the topic names and brief 1-sentence description of the topic
- Topics can include:
- AI tools
- GPT Models
- Google Models
- LLMs
- llama Models
- Falcon Models
- Programming Languages
- AI recent News
- AI tutorials
- OpenAI
- AI for business 
- AI for education
- AI for medicine 
- AI for art and music
- Deep Learning
- NLP
- Machine Learning
- Data science
- Opportunities in AI
- AI frameworks
- Future AI
- Langchain

- Provide a brief description of the topics after the topic name. Example: 'Topic: Brief Description'
- Use the same words and terminology that is said in the websites' content
- Do not respond with numbers, just bullet points of all topics listed under each other. Example:
Topics:
    - Topic 1 title: topic 1 description
   

In [44]:
print(topics_found)

- Democracies face 'truth decay' as AI blurs fact and fiction: AI technology causing decline in ability to distinguish truth and falsehoods in democratic societies.
- Tech leaders agree on AI regulation but divided on how in Washington forum: Consensus on need for AI regulation, disagreement on specific approach.
- Paedophiles using open source AI to create child sexual abuse content: Open source AI exploited by paedophiles to generate and distribute child sexual abuse material.
- Microsoft president and Nvidia chief scientist testify in Senate AI hearings: Testimony in Senate hearings on impact and regulation of AI technology.
- Self-publishers must declare if content sold on Amazon's site is AI-generated: Requirement for self-publishers on Amazon to disclose if content is generated using AI technology.
- The strain of moderating social media: Moderating social media platforms taking toll on mental health and well-being of moderators.
- Ignorance is not bliss for ChatGPT: AI language 

In [14]:
print(type(topics_found))

<class 'str'>


In [15]:
!pip install kor


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:

from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

In [46]:
schema = Object(
    id="topic",
    description="Topic Information",
    examples=[
        ("Artificial intelligence: MIT News covers advancements, applications, and research in AI technology and algorithms.", [{"title": "Artificial intelligence"}, {"description": "MIT News covers advancements, applications, and research in AI technology and algorithms."}, {"tag": "AI News"}]),
        ("Generative AI: An exciting tool in AI that can generate text based on input prompts.", [{"title": "Generative AI"}, {"description": "An exciting tool in AI that can generate text based on input prompts."}, {"tag": "AI Tools"}]),
        ("AI in relationships: The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app.", [{"title": "AI in relationships"}, {"description": "The use of AI in relationship coaching and mentoring, such as the example of an AI-powered romantic relationship coaching app."}, {"tag": "AI Applications"}]),
        ("AI in different industries: Exploring the potential of AI in various industries beyond consumer software and the internet.", [{"title": "AI in different industries"}, {"description": "Exploring the potential of AI in various industries beyond consumer software and the internet."}, {"tag": "AI Opportunities"}]),
        ("Supervised Learning: A technique in AI that is good at labeling things or computing input to outputs.", [{"title": "Supervised Learning"}, {"description": "A technique in AI that is good at labeling things or computing input to outputs."}, {"tag": "AI Tools"}]),
        ("Large Language Models: The power and potential of large language models in AI applications.", [{"title": "Large Language Models"}, {"description": "The power and potential of large language models in AI applications."}, {"tag": "AI LLMs"}]),
        ("Future growth of AI technologies: The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term.", [{"title": "Future growth of AI technologies:"}, {"description": "The prediction that supervised learning and generative AI will continue to grow in value and adoption over the next three years, with the potential for even greater expansion in the long term."}, {"tag": "Future of AI"}]),
        ("Large Language Models: Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently.", [{"title": "Large Language Models:"}, {"description": " Dr. Andrew explains how large language models, like GPT, are built using supervised learning to predict the next word, enabling applications that can be built faster and more efficiently."}, {"tag": "AI LLMs"}]),
 
    ],
    attributes=[
         Text(
            id="title",
            description="The title of the topic listed",
        ),
        Text(
            id="description",
            description="The description of the topic listed",
        ),
        Text(
            id="tag",
            description="The type of content being described",
        )
    ],
    many=True,
)

In [47]:
#chain = create_extraction_chain(schema, llm3)
chain = create_extraction_chain(llm3, schema)

In [48]:
topics_structured = chain.run(topics_found)

In [49]:
topics_structured

{'data': {'topic': [{'title': "Democracies face 'truth decay' as AI blurs fact and fiction",
    'description': '',
    'tag': 'AI Impact'},
   {'title': 'Tech leaders agree on AI regulation but divided on how in Washington forum',
    'description': '',
    'tag': 'AI Regulation'},
   {'title': 'Paedophiles using open source AI to create child sexual abuse content',
    'description': '',
    'tag': 'AI Misuse'},
   {'title': 'Microsoft president and Nvidia chief scientist testify in Senate AI hearings',
    'description': '',
    'tag': 'AI Testimony'},
   {'title': "Self-publishers must declare if content sold on Amazon's site is AI-generated",
    'description': '',
    'tag': 'AI Disclosure'},
   {'title': 'The strain of moderating social media',
    'description': '',
    'tag': 'AI Impact'},
   {'title': 'Ignorance is not bliss for ChatGPT',
    'description': '',
    'tag': 'AI Bias'},
   {'title': 'The future out of our control',
    'description': '',
    'tag': 'AI Concerns'

# Expand on Topics Found

In [50]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n", " "], chunk_size=4000, chunk_overlap=800)

docs= text_splitter.split_documents(website_data)

In [51]:
openai_embeddings = OpenAIEmbeddings(show_progress_bar=True, embedding_ctx_length=1024)

In [52]:
!pip install faiss-cpu


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
website_db = FAISS.from_documents(docs, openai_embeddings)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.49s/it]


In [54]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [55]:
from langchain.chains import RetrievalQA , RetrievalQAWithSourcesChain

In [56]:

# I'm also setting k=4 so the number of relevant docs we get back is 4. 
qa = RetrievalQA.from_chain_type(llm=llm3,
                                 chain_type="stuff",
                                 retriever=website_db.as_retriever(k=4),
                                 chain_type_kwargs = {
                                     'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 },
                                return_source_documents=True)
                                 

In [57]:
topics_structured= topics_structured["data"]["topic"]

In [58]:
topics_structured

[{'title': "Democracies face 'truth decay' as AI blurs fact and fiction",
  'description': '',
  'tag': 'AI Impact'},
 {'title': 'Tech leaders agree on AI regulation but divided on how in Washington forum',
  'description': '',
  'tag': 'AI Regulation'},
 {'title': 'Paedophiles using open source AI to create child sexual abuse content',
  'description': '',
  'tag': 'AI Misuse'},
 {'title': 'Microsoft president and Nvidia chief scientist testify in Senate AI hearings',
  'description': '',
  'tag': 'AI Testimony'},
 {'title': "Self-publishers must declare if content sold on Amazon's site is AI-generated",
  'description': '',
  'tag': 'AI Disclosure'},
 {'title': 'The strain of moderating social media',
  'description': '',
  'tag': 'AI Impact'},
 {'title': 'Ignorance is not bliss for ChatGPT',
  'description': '',
  'tag': 'AI Bias'},
 {'title': 'The future out of our control',
  'description': '',
  'tag': 'AI Concerns'},
 {'title': 'How savvy trillion-dollar chipmaker Nvidia is powe

In [59]:
tweets=[]
for topic in topics_structured:

            topic_title=topic['title']
            topic_summary=""
            topic_source=""
            topic_publish_date=""

            query = f"""
                {topic['title']}: {topic['description']}
            """
            #expanded_topic = qa.run(query)
            expanded_topic = qa({"query": query})
            print(f"{topic['title']}: {topic['description']}")
            print ("\n\n")
            print(expanded_topic['result'])
            print ("\n\n")
            print(expanded_topic['source_documents'])
            print ("\n\n")


            topic_summary=expanded_topic['result']
            topic_source=expanded_topic['source_documents'][0].metadata['source']
            print(topic_source)

            tweet_details = {"topic_title":topic_title, "topic_summary": topic_summary, "topic_source":topic_source}
                             #, "topic_publish_date":""}
            tweets.append(tweet_details)

100%|██████████| 1/1 [00:02<00:00,  2.42s/it]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Crosswords
                        



                            Wordiply
                        



                            Corrections
                        









                        Facebook
                    









                        Twitter
                    





            Search jobs
        



            Holidays
        



            Digital Archive
        



            Guardian Puzzles app
        



            Guardian Licensing
        








                    News
       

100%|██████████| 1/1 [00:00<00:00,  1.02it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Crosswords
                        



                            Wordiply
                        



                            Corrections
                        









                        Facebook
                    









                        Twitter
                    





            Search jobs
        



            Holidays
        



            Digital Archive
        



            Guardian Puzzles app
        



            Guardian Licensing
        








                    News
       

100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


...



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Crosswords
                        



                            Wordiply
                        



                            Corrections
                        









                        Facebook
                    









                        Twitter
                    





            Search jobs
        



            Holidays
        



            Digital Archive
        



            Guardian Puzzles app
        



            Guardian Licensing
        








                    News
       

100%|██████████| 1/1 [00:01<00:00,  1.13s/it]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Crosswords
                        



                            Wordiply
                        



                            Corrections
                        









                        Facebook
                    









                        Twitter
                    





            Search jobs
        



            Holidays
        



            Digital Archive
        



            Guardian Puzzles app
        



            Guardian Licensing
        








                    News
       

100%|██████████| 1/1 [00:00<00:00,  1.37it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



In [60]:
tweets

[{'topic_title': "Democracies face 'truth decay' as AI blurs fact and fiction",
  'topic_summary': 'The head of Australia\'s military, Gen Angus Campbell, has warned that democracies are facing a phenomenon called "truth decay" as artificial intelligence (AI) blurs the lines between fact and fiction. Disinformation campaigns from countries like Russia could erode trust and sow discord in the western world. Tech leaders have agreed on the need for AI regulation, but are divided on how to achieve it. There are also concerns about the use of open source AI by paedophiles to create child sexual abuse content. Microsoft and Nvidia have testified in Senate AI hearings, and self-publishers on Amazon must now declare if their content is AI-generated.',
  'topic_source': 'https://www.theguardian.com/technology/artificialintelligenceai'},
 {'topic_title': 'Tech leaders agree on AI regulation but divided on how in Washington forum',
  'topic_summary': 'Tech leaders recently gathered in Washington

In [30]:
for topic in topics_structured:
    query = f"""
        {topic['title']}: {topic['description']}
    """
    #expanded_topic = qa.run(query)
    expanded_topic = qa({"query": query})
    print(f"{topic['title']}: {topic['description']}")
    print ("\n\n")
    print(expanded_topic['result'])
    print ("\n\n")
    print(expanded_topic['source_documents'])
    print ("\n\n")

100%|██████████| 1/1 [00:00<00:00,  2.59it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  2.44it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


...



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:00<00:00,  2.45it/s]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



100%|██████████| 1/1 [00:01<00:00,  1.42s/it]




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You will be given text from websites content which contains many topics.
You goal is to write a summary (5 sentences or less) about a topic the user chooses
The summary should be relative to the description assossiated with the Topic Description
Do not respond with information that isn't relevant to the topic that the user gives you
----------------
Artificial intelligence | MIT News | Massachusetts Institute of Technology



 






Skip to content ↓











Massachusetts Institute of Technology



MIT Top Menu↓

Education
Research
Innovation
Admissions + Aid
Campus Life
News
Alumni
About MIT

More ↓












Search MIT







Search websites, locations, and people

 















See More Results


Suggestions or feedback?















































MIT News | Massachusetts Institute of Technology












Subscribe to MIT News newsletter



In [31]:


# for topic in topics_structured[:1]:
#     query = f"""
#         {topic['title']}: {topic['description']}
#     """
#     #expanded_topic = qa.run(query)
#     expanded_topic = qa({"query": query})
#     print(f"{topic['title']}: {topic['description']}")
#     print ("\n\n")
#     print(expanded_topic['result'])
#     print ("\n\n")
#     print(type(expanded_topic['source_documents'][0]))  # document
#     print(type(expanded_topic['source_documents']))  #list
#     print ("\n\n")

#     for topic in expanded_topic['source_documents']:
#         print(type(topic)) #document
#         print(topic.metadata['source'])

    
        

# print(expanded_topic['source_documents'][0].metadata['source'])

# youtube_video_ID=expanded_topic['source_documents'][0].metadata['source']
# youtube_video_URL=f"https://www.youtube.com/watch?v={youtube_video_ID}"
# print(youtube_video_URL)

# #docs[0].metadata['source']